## 0)import libraries 

In [1]:
"""
Thesis_PhD_Qianqian Predicted_fluxes_glboal_stripes
date: 24-Feb-2023
author: Qianqian
Contact: q.han@utwente.nl
-------------------------------------
Description: 
"""
# libraries
import os
import joblib
from osgeo import gdal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from rasterio.warp import reproject, Resampling
import xarray as xr
import pickle
from PyStemmusScope import variable_conversion as vc
from rasterio.warp import reproject, Resampling
from sklearn.preprocessing import OneHotEncoder
import time
import re
import glob
import rioxarray
import dask.array
from dask.distributed import Client, progress

## 1)settings for Dask

In [2]:
# Set the MALLOC_TRIM_THRESHOLD_ environment variable
os.environ['MALLOC_TRIM_THRESHOLD_'] = '0'

# Run your Dask script or notebook
...

Ellipsis

In [3]:
# francesco
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(cores=16, memory='240GiB', queue='fat', processes=8, death_timeout=600, local_directory='$TMPDIR', walltime='5:00:00')
cluster.scale(jobs=2)

client = Client(cluster) 
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://145.136.63.47:45055,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
# client = Client(n_workers=4, threads_per_worker=2)
# client

## 2)define working path, load trained model, define functions

In [67]:
workingPath = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/"

In [68]:
# load trained RF model, better use not parallel model with Dask
with open(workingPath+'3RF_train/RFLEH-LAI-hc-CO2-SSM-Vcmo-IGBP_multi7_1core_snellius0629.pkl', 'rb') as f:
    rfLEHmulti = pickle.load(f)

In [69]:
# function for loading the trained model
def load_model(path):
    # load trained RF model, better use not parallel model with Dask
    with open(path, 'rb') as f:
        rfLEHmulti = pickle.load(f)
    return rfLEHmulti

In [70]:
# function for getting directory of input data
def get_directories_with_number_and_ending(directory_path, ending):
    directories = []
    if ending:
        pattern = re.compile(r'^\d+.*{}$'.format(re.escape(ending)))  # Match directory names with number at the beginning and specific ending
    else:
        pattern = re.compile(r'^\d+$')  # Match directory names with only numbers

    for entry in os.scandir(directory_path):
        if entry.is_dir():
            directory_name = entry.name
            if pattern.match(directory_name):
                directories.append(directory_name)

    return directories

## 3) read data

In [74]:
### define the path of input data, set the reference file for spatial resample
inputData = workingPath+"1input_data/"
# get all the filefolders named as year
year_list = get_directories_with_number_and_ending(workingPath+"1input_data/", "global")
# the input data from year[0] to year[..], based on the process id in sbatch script
year = year_list[4-1]
print(year)
# # reference file for spatial resample
# era5 = xr.open_dataset(inputData+"/2015global/era5land/era5-land_10m_u_component_of_wind_2015-01.nc")#.sel(latitude=slice(55, 45), longitude=slice(2, 12))#.sel(time=t, method='nearest')['u10']

2015global


In [75]:
lat1 = 40
lat2 = 55
lon1 = 2
lon2 = 17

In [80]:
all1 = xr.open_dataset(inputData+year+"/era5land/era5land2015_10km1.nc", chunks={'longitude': 50, 'latitude': 50})

In [81]:
all1 = all1.sel(
    latitude=slice(lat2,lat1),  
    longitude=slice(lon1, lon2)  
)

In [82]:
all_resample = all1.resample(time="1800S").interpolate('linear')
all_resample = all_resample.isel(time=slice(0,17000))

In [83]:
%%time
# calculate the Rin and Rli difference for every hour
Rin = all1['ssrd'].diff("time")/3600  #xr.concat([all1['ssrd'].isel(time=0),all1['ssrd']], dim="time")
Rin[0::24] = all1['ssrd'][1::24]/3600 # assign the original values in t01
# Rin.mean(dim=['longitude','latitude']).compute()[48]
Rli = all1['strd'].diff("time")/3600  
Rli[0::24] = all1['strd'][1::24]/3600
# Rli.mean(dim=['longitude','latitude']).plot()
Rin = Rin.resample(time="1800S").interpolate('linear')
Rli = Rli.resample(time="1800S").interpolate('linear')
Rin

CPU times: user 63.5 ms, sys: 110 µs, total: 63.6 ms
Wall time: 61.5 ms


<xarray.DataArray 'ssrd' (time: 17517, latitude: 151, longitude: 151)>
dask.array<transpose, shape=(17517, 151, 151), dtype=float32, chunksize=(17517, 50, 50), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 55.0 54.9 54.8 54.7 ... 40.3 40.2 40.1 40.0
  * longitude  (longitude) float32 2.0 2.1 2.2 2.3 2.4 ... 16.7 16.8 16.9 17.0
  * time       (time) datetime64[ns] 2015-01-01T01:00:00 ... 2015-12-31T23:00:00

In [84]:
all1.nbytes/2**30

5.952685706317425

In [85]:
### 1) read LAI data
lai = xr.open_dataset(inputData+"/2015global/lai/LAI_global2015_10km.nc", chunks={'longitude': 50, 'latitude': 50})['LAI']

In [96]:
inputData+"/2015global/lai/LAI_global2015_10km.nc"

'/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data//2015global/lai/LAI_global2015_10km.nc'

In [86]:
lai = lai.sel(
    latitude=slice(lat2,lat1), 
    longitude=slice(lon1, lon2)  
)

In [87]:
lai = lai.resample(time="1800S").interpolate('linear')

In [88]:
lai

<xarray.DataArray 'LAI' (time: 12625, latitude: 151, longitude: 151)>
dask.array<transpose, shape=(12625, 151, 151), dtype=float32, chunksize=(12625, 50, 50), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 2.0 2.1 2.2 2.3 2.4 ... 16.7 16.8 16.9 17.0
  * latitude   (latitude) float32 55.0 54.9 54.8 54.7 ... 40.3 40.2 40.1 40.0
  * time       (time) datetime64[ns] 2015-01-03 ... 2015-09-23
Attributes:
    valid_range:    [  0 210]
    flag_values:    [253 254]
    flag_meanings:  Out_of_range_superior_to_physical_max Out_of_range_inferi...
    long_name:      Leaf Area Index 1km
    standard_name:  leaf_area_index
    units:          
    grid_mapping:   crs

In [98]:
### 2) read canopy height data
hc_path = inputData+year+'/canopy_height/canopy_height_11kmEurope20230921_10km.nc'
hc = xr.open_dataset(hc_path, chunks={'x': 50, 'y': 50}).rename({'x':'longitude','y':'latitude'})
hc

<xarray.Dataset>
Dimensions:                        (longitude: 1002, latitude: 469, band: 1)
Coordinates:
  * longitude                      (longitude) float32 -31.2 -31.1 ... 68.8 68.9
  * latitude                       (latitude) float32 81.8 81.7 ... 35.1 35.0
  * band                           (band) int64 1
Data variables:
    spatial_ref                    int64 ...
    __xarray_dataarray_variable__  (band, latitude, longitude) float32 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>

In [94]:
hc = hc.sel(
    latitude=slice(lat2+0.01,lat1),  
    longitude=slice(lon1-0.01, lon2)  
)
hc

<xarray.DataArray '__xarray_dataarray_variable__' (band: 1, latitude: 151,
                                                   longitude: 151)>
dask.array<getitem, shape=(1, 151, 151), dtype=float32, chunksize=(1, 50, 50), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 1
  * longitude    (longitude) float64 2.0 2.1 2.2 2.3 2.4 ... 16.7 16.8 16.9 17.0
  * latitude     (latitude) float64 55.0 54.9 54.8 54.7 ... 40.3 40.2 40.1 40.0
    spatial_ref  int64 0
Attributes:
    NETCDF_DIM_band_DEF:     [ 1. 10.]
    NETCDF_DIM_band_VALUES:  1
    NETCDF_DIM_EXTRA:        {band}
    AREA_OR_POINT:           Area
    descriptions:            b1
    offsets:                 0
    scales:                  1
    _FillValue:              nan
    scale_factor:            1.0
    add_offset:              0.0

In [63]:
### 3) read CO2 data
## resampled CO2
ds_co2_10km = xr.open_dataset(inputData+year+"/co2/CAMS_CO2_2015_10km.nc", chunks="auto")['co2']
ds_co2_10km = ds_co2_10km.sel(
    latitude=slice(lat2,lat1),  
    longitude=slice(lon1, lon2)  
)

In [64]:
ds_co2_10km

<xarray.DataArray 'co2' (time: 2920, latitude: 151, longitude: 151)>
dask.array<getitem, shape=(2920, 151, 151), dtype=float32, chunksize=(848, 136, 151), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 2.0 2.1 2.2 2.3 2.4 ... 16.7 16.8 16.9 17.0
  * latitude   (latitude) float32 55.0 54.9 54.8 54.7 ... 40.3 40.2 40.1 40.0
  * time       (time) datetime64[ns] 2015-01-01T00:30:00 ... 2015-12-31T21:30:00
Attributes:
    grid_mapping:  spatial_ref

In [65]:
ds_co2_10km = ds_co2_10km.resample(time="1800S").interpolate('linear')

In [66]:
ds_co2_10km

<xarray.DataArray 'co2' (time: 17515, latitude: 151, longitude: 151)>
dask.array<transpose, shape=(17515, 151, 151), dtype=float32, chunksize=(17515, 136, 151), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 2.0 2.1 2.2 2.3 2.4 ... 16.7 16.8 16.9 17.0
  * latitude   (latitude) float32 55.0 54.9 54.8 54.7 ... 40.3 40.2 40.1 40.0
  * time       (time) datetime64[ns] 2015-01-01T00:30:00 ... 2015-12-31T21:30:00
Attributes:
    grid_mapping:  spatial_ref

In [29]:
### 4) read the resampled SSM data
## netcdf file works well
path_SSM = glob.glob(inputData+year+"/ssm/SM2015Europe11kmEurope20230921_10km.nc")[0]
ds_SSM = xr.open_dataset(path_SSM)['__xarray_dataarray_variable__'].rename('SSM')
# ds_SSM = ds_SSM.sortby(["longitude", "latitude"])
ds_SSM = ds_SSM.sel(
    latitude=slice(lat2,lat1),  
    longitude=slice(lon1, lon2)  
)
SSM = ds_SSM.resample(time="1800S").interpolate('linear')/1000 

In [30]:
SSM

<xarray.DataArray 'SSM' (time: 17473, latitude: 151, longitude: 151)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ..., 0.28149081,
                nan,        nan],
        [       nan,        nan,        nan, ..., 0.279     ,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [       nan,        nan,        nan, ..., 0.25348486,
                nan,        nan],
        [       nan,        nan,        nan, ..., 0.249375  ,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ..., 0.25398532,
                nan,        nan],
        [       nan,        nan,        nan, ..., 0.25      ,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]])
Coordinates:
  * longitude  (longitude) float32 2.0 2.1 2.2 2.3 2.4 ... 16.7 16.8 16.9 17.0
  * latitude   (latitude) float32 55.0 54.9 54.8 54.7 ... 40.3 40.2 40.1 40.0
  * time       (time) datetime64[ns] 2015-01-01 ... 2015-12-31

In [31]:
### 5) read Vcmax data
ds_Vcmo = xr.open_rasterio(inputData+year+"/vcmax/TROPOMI_Vmax_Tg_mean10km_global.nc", engine="rasterio").rename({'x':'longitude','y':'latitude'})

/scratch-local/fnattino30.3957759/ipykernel_1457079/170410605.py:2: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  ds_Vcmo = xr.open_rasterio(inputData+year+"/vcmax/TROPOMI_Vmax_Tg_mean10km_global.nc", engine="rasterio").rename({'x':'longitude','y':'latitude'})


In [33]:
ds_Vcmo = ds_Vcmo.sel(longitude=Rin.longitude, latitude=Rin.latitude, method='nearest', tolerance=0.01)

In [34]:
ds_Vcmo

<xarray.DataArray (band: 1, latitude: 151, longitude: 151)>
array([[[      nan,       nan, ...,       nan,  8.572168],
        [      nan,       nan, ...,       nan,  8.572168],
        ...,
        [      nan,       nan, ..., 24.238039, 18.044699],
        [20.559444, 20.559444, ..., 21.088354, 18.009357]]], dtype=float32)
Coordinates:
  * band       (band) int64 1
  * latitude   (latitude) float64 55.0 54.9 54.8 54.7 ... 40.3 40.2 40.1 40.0
  * longitude  (longitude) float64 2.0 2.1 2.2 2.3 2.4 ... 16.7 16.8 16.9 17.0
Attributes:
    transform:   (0.09999999830410791, 0.0, -179.94999389563642, 0.0, -0.1, 9...
    crs:         +init=epsg:4326
    res:         (0.09999999830410791, 0.1)
    is_tiled:    0
    nodatavals:  (nan,)
    scales:      (1.0,)
    offsets:     (0.0,)

In [103]:
### 6) read IGBP data
# read lan
path_landcover = inputData+year+"/igbp/landcover10km_global.nc"
landcover = xr.open_rasterio(path_landcover, engine="rasterio", chunks="auto").rename({'x':'longitude','y':'latitude'})
landcover = landcover.where(longitude=Rin.longitude, latitude=Rin.latitude, method='nearest', tolerance=0.01)

/scratch-local/fnattino30.3957759/ipykernel_1457079/467170179.py:4: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  landcover = xr.open_rasterio(path_landcover, engine="rasterio", chunks="auto").rename({'x':'longitude','y':'latitude'})


Signature: landcover.where(cond: 'Any', other: 'Any' = <NA>, drop: 'bool' = False) -> 'T_DataWithCoords'
Docstring:
Filter elements from this object according to a condition.

This operation follows the normal broadcasting and alignment rules that
xarray uses for binary arithmetic.

Parameters
----------
cond : DataArray, Dataset, or callable
    Locations at which to preserve this object's values. dtype must be `bool`.
    If a callable, it must expect this object as its only parameter.
other : scalar, DataArray or Dataset, optional
    Value to use for locations in this object where ``cond`` is False.
    By default, these locations filled with NA.
drop : bool, default: False
    If True, coordinate labels that only correspond to False values of
    the condition are dropped from the result.

Returns
-------
DataArray or Dataset
    Same xarray type as caller, with dtype float64.

Examples
--------
>>> a = xr.DataArray(np.arange(25).reshape(5, 5), dims=("x", "y"))
>>> a
<xarray.DataA

In [102]:
landcover

<xarray.DataArray (band: 1, latitude: 0, longitude: 0, time: 17517)>
dask.array<where, shape=(1, 0, 0, 17517), dtype=float64, chunksize=(1, 0, 0, 17517), chunktype=numpy.ndarray>
Coordinates:
  * band       (band) int64 1
  * latitude   (latitude) float64 
  * longitude  (longitude) float64 
  * time       (time) datetime64[ns] 2015-01-01T01:00:00 ... 2015-12-31T23:00:00
Attributes:
    transform:   (0.09999999830410791, 0.0, -179.94999389563642, 0.0, -0.1, 9...
    crs:         +init=epsg:4326
    res:         (0.09999999830410791, 0.1)
    is_tiled:    0
    nodatavals:  (255.0,)
    scales:      (1.0,)
    offsets:     (0.0,)

In [36]:
landcover

<xarray.DataArray (band: 1, latitude: 151, longitude: 151)>
dask.array<getitem, shape=(1, 151, 151), dtype=uint8, chunksize=(1, 151, 151), chunktype=numpy.ndarray>
Coordinates:
  * band       (band) int64 1
  * latitude   (latitude) float64 55.0 54.9 54.8 54.7 ... 40.3 40.2 40.1 40.0
  * longitude  (longitude) float64 2.0 2.1 2.2 2.3 2.4 ... 16.7 16.8 16.9 17.0
Attributes:
    transform:   (0.09999999830410791, 0.0, -179.94999389563642, 0.0, -0.1, 9...
    crs:         +init=epsg:4326
    res:         (0.09999999830410791, 0.1)
    is_tiled:    0
    nodatavals:  (255.0,)
    scales:      (1.0,)
    offsets:     (0.0,)

In [37]:
# read IGBP unique values
data = pd.read_csv(inputData+'IGBP11unique.csv')['0'].unique()
# read the table for converting landcover to IGBP
IGBP_table = pd.read_csv(inputData+year+"/igbp/lccs_to_igbp_table.csv")

In [38]:
def landcover_to_igbp(landcover, IGBP_table, data):
    IGBP = np.vectorize(IGBP_table.set_index("lccs_class").T.to_dict('records')[0].get)(landcover.values)
    IGBP_all = pd.DataFrame(columns=['IGBP_veg_long1', 'IGBP_veg_long2', 'IGBP_veg_long3','IGBP_veg_long4','IGBP_veg_long5',
                              'IGBP_veg_long6','IGBP_veg_long7','IGBP_veg_long8','IGBP_veg_long9',
                             'IGBP_veg_long10','IGBP_veg_long11'])
    # define one hot encoding for IGBP
    encoder = OneHotEncoder(categories=[data]*1,sparse=False,
                           handle_unknown = "ignore")
    # transform data
    aa = encoder.fit_transform(IGBP.reshape(IGBP.shape[0]*IGBP.shape[1],1))
    # assign 23-D IGBP into 23 columns
    for i in range(1,12,1):
        IGBP_all['IGBP_veg_long'+str(i)] = aa[:,i-1]
    return IGBP_all

## 4) chunk all the input variables

In [44]:
all_resample

<xarray.Dataset>
Dimensions:    (latitude: 151, time: 17000, longitude: 151)
Coordinates:
  * latitude   (latitude) float32 55.0 54.9 54.8 54.7 ... 40.3 40.2 40.1 40.0
  * longitude  (longitude) float32 2.0 2.1 2.2 2.3 2.4 ... 16.7 16.8 16.9 17.0
  * time       (time) datetime64[ns] 2015-01-01 ... 2015-12-21T03:30:00
Data variables:
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(17000, 14, 70), meta=np.ndarray>
    sp         (time, latitude, longitude) float32 dask.array<chunksize=(17000, 14, 70), meta=np.ndarray>
    ssrd       (time, latitude, longitude) float32 dask.array<chunksize=(17000, 14, 70), meta=np.ndarray>
    strd       (time, latitude, longitude) float32 dask.array<chunksize=(17000, 14, 70), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(17000, 14, 70), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(17000, 14, 70), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(17000, 14, 70), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(17000, 14, 70), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-19 23:48:36 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [39]:
ds_era5land = all_resample#.sel(time=t, method='nearest')#.compute()

ds_ss = xr.Dataset()
ds_ss["Rin"] = Rin.isel(time=slice(0,17000)).chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)})#.sel(time=t, method='nearest')#.compute()
ds_ss["Rli"] = Rli.isel(time=slice(0,17000)).chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)})#.sel(time=t, method='nearest')#.compute()
ds_ss["p"] = all_resample["sp"].chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)})/100   # Pa -> hPa
ds_ss["Ta"] = all_resample["t2m"].chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)}) - 273.15  # K -> degC
ds_ss["ea"] = vc.calculate_es(all_resample["d2m"] - 273.15).chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)})
ds_ss["u"] = (all_resample["u10"].chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)}) ** 2 + all_resample["v10"].chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)}) ** 2) ** 0.5
ds_ss["Precip_msr"] = all_resample["tp"].chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)})*1000   # mm
ds_ss['LAI'] = lai.chunk()#.sel(time=t, method='nearest')#.compute()
ds_ss['CO2'] = ds_co2_10km.chunk()#.sel(time=t, method='nearest')#.compute()
ds_ss['SSM'] = SSM.chunk()#.sel(time=t, method='nearest')#.compute()

ds_ss = ds_ss.assign_coords({"longitude": hc.longitude.values, "latitude": hc.latitude.values})
ds_Vcmo = ds_Vcmo.assign_coords({"longitude": hc.longitude.values, "latitude": hc.latitude.values})
landcover = landcover.assign_coords({"longitude": hc.longitude.values, "latitude": hc.latitude.values})

ds_ss = ds_ss.isel(time=slice(0,17000))


/gpfs/home2/fnattino30/mambaforge/envs/emulator/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/gpfs/home2/fnattino30/mambaforge/envs/emulator/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]

In [41]:
ds_ss['Rin']

<xarray.DataArray 'Rin' (time: 17000, latitude: 151, longitude: 151)>
dask.array<rechunk-merge, shape=(17000, 151, 151), dtype=float32, chunksize=(17000, 151, 151), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 55.0 54.9 54.8 54.7 ... 40.3 40.2 40.1 40.0
  * longitude  (longitude) float64 2.0 2.1 2.2 2.3 2.4 ... 16.7 16.8 16.9 17.0
  * time       (time) datetime64[ns] 2015-01-01T01:00:00 ... 2015-12-21T04:30:00

In [39]:
ds_ss.nbytes/2**30

18.77194495499134

## 5) predict fluxes with map_blocks

In [40]:
ds_ss = ds_ss.to_array()

In [41]:
# ds_ss = ds_ss.chunk({"time":2000, "variable":10})

In [49]:
ds_ss = ds_ss.chunk({"time":1000, "variable":10, "latitude":51, "longitude":51})

In [50]:
hc=hc.chunk({"latitude":51, "longitude":51}).squeeze()
ds_Vcmo = ds_Vcmo.chunk({"latitude":51, "longitude":51}).squeeze()
landcover = landcover.chunk({"latitude":51, "longitude":51}).squeeze()

In [51]:
chunks = [ds_ss.chunksizes[v] for v in ['time', 'latitude', 'longitude']]
chunks.append((7,))

In [52]:
estimated_LEH = xr.DataArray(
    name = 'LEH',
    data=dask.array.zeros((len(ds_ss.time), len(ds_ss.latitude), len(ds_ss.longitude),7), chunks=chunks),
    dims=("time", "latitude", "longitude","variable_out"),
    coords={
        "variable_out":range(7), 
        "time":ds_ss.time, 
        "latitude":ds_ss.latitude,
        "longitude":ds_ss.longitude
    }
)

In [53]:
def predictFlux(ds_ss, hc, ds_Vcmo, landcover, IGBP_table, training_testing_append, path_model):
    # ds_ss = ds_ss.reshape(10,-1)
    # time = ds_ss.time
    # ds_ss = ds_ss.squeeze("time")
    hc = hc.sel(latitude=ds_ss.latitude, longitude=ds_ss.longitude)
    hc_line = np.tile(hc.to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size) #如果和动态变量一样放进ds_ss里显示为一些离散的点，不知道为什么，所以静态变量放在时间循环外面
    print(hc_line.shape)
    ds_Vcmo = ds_Vcmo.sel(latitude=ds_ss.latitude, longitude=ds_ss.longitude)
    Vcmo_line = np.tile(ds_Vcmo.to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    landcover = landcover.sel(latitude=ds_ss.latitude, longitude=ds_ss.longitude)
    IGBP_all = landcover_to_igbp(landcover, IGBP_table, training_testing_append)
    IGBP1 = np.tile(IGBP_all.iloc[:,0].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP2 = np.tile(IGBP_all.iloc[:,1].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP3 = np.tile(IGBP_all.iloc[:,2].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP4 = np.tile(IGBP_all.iloc[:,3].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP5 = np.tile(IGBP_all.iloc[:,4].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP6 = np.tile(IGBP_all.iloc[:,5].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP7 = np.tile(IGBP_all.iloc[:,6].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP8 = np.tile(IGBP_all.iloc[:,7].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP9 = np.tile(IGBP_all.iloc[:,8].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP10 = np.tile(IGBP_all.iloc[:,9].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP11 = np.tile(IGBP_all.iloc[:,10].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    
    
    Rin_line = ds_ss[0,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    Rli_line = ds_ss[1,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    p_line = ds_ss[2,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    Ta_line = ds_ss[3,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    ea_line = ds_ss[4,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    u_line = ds_ss[5,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    Precip_msr_line = ds_ss[6,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    LAI_line = ds_ss[7,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    CO2_line = ds_ss[8,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    SSM_line = ds_ss[9,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)

    features_arr = np.concatenate((Rin_line, Rli_line, p_line, Ta_line,ea_line, u_line, Precip_msr_line,
                                   LAI_line, hc_line, CO2_line, SSM_line, Vcmo_line,
                                   IGBP1, IGBP2, IGBP3,IGBP4,IGBP5,IGBP6,IGBP7,IGBP8,IGBP9,IGBP10,IGBP11))
    features_arr = features_arr.transpose()
    # Nan value.
    df_features = pd.DataFrame(data=features_arr)
    df_features_drop_nan = df_features.dropna()
    invalid_index = sorted(set(df_features.index.to_list()) - set(df_features_drop_nan.index.to_list()))

    # # Convert the nan value as 0 for the calculation
    where_are_NaNs = np.isnan(features_arr)
    features_arr[where_are_NaNs] = 0
    
    model = load_model(path_model)
    
    # model = path_model
    estimated_LEH = model.predict(features_arr)
    estimated_LEH[invalid_index] = np.nan
    LEH_map = estimated_LEH.reshape(ds_ss.time.size, ds_ss.latitude.size, ds_ss.longitude.size,7)
    
    return xr.DataArray(
        name = 'LEH',
        data=LEH_map,
        dims=("time", "latitude", "longitude","variable_out"),
        coords={
            "variable_out":range(7), 
            "time":ds_ss.time, 
            "latitude":ds_ss.latitude,
            "longitude":ds_ss.longitude
        }
    )
result = xr.map_blocks(
    predictFlux,
    ds_ss,
    args=[hc, ds_Vcmo, landcover],
    kwargs={"IGBP_table":IGBP_table, "training_testing_append":data, "path_model":'/gpfs/work1/0/einf2480/global_data_Qianqian/RF_train/RFLEH-LAI-hc-CO2-SSM-Vcmo-IGBP_multi7_1core_snellius0629.pkl'},
    template=estimated_LEH,
)

In [54]:
result

<xarray.DataArray 'LEH' (time: 17000, latitude: 151, longitude: 151,
                         variable_out: 7)>
dask.array<<this-array>-predictFlux, shape=(17000, 151, 151, 7), dtype=float64, chunksize=(1000, 51, 51, 7), chunktype=numpy.ndarray>
Coordinates:
  * variable_out  (variable_out) int64 0 1 2 3 4 5 6
  * time          (time) datetime64[ns] 2015-01-01T01:00:00 ... 2015-12-21T04...
  * latitude      (latitude) float64 55.0 54.9 54.8 54.7 ... 40.3 40.2 40.1 40.0
  * longitude     (longitude) float64 2.0 2.1 2.2 2.3 ... 16.7 16.8 16.9 17.0

In [55]:
start_time = time.time()  # Record the start time
result1 = result.compute()
end_time = time.time()  # Record the end time
total_running_time = end_time - start_time
print("Total Running Time:", total_running_time)

2023-09-21 10:17:56,221 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/home/qiahan/.conda/envs/firstEnv/lib/python3.9/site-packages/distributed/protocol/core.py", line 158, in loads
    return msgpack.loads(
  File "msgpack/_unpacker.pyx", line 194, in msgpack._cmsgpack.unpackb
ValueError: 2302901000 exceeds max_bin_len(2147483647)
2023-09-21 10:17:56,224 - distributed.core - ERROR - Exception while handling op register-client
Traceback (most recent call last):
  File "/home/qiahan/.conda/envs/firstEnv/lib/python3.9/site-packages/distributed/core.py", line 922, in _handle_comm
    result = await result
  File "/home/qiahan/.conda/envs/firstEnv/lib/python3.9/site-packages/distributed/scheduler.py", line 5386, in add_client
    await self.handle_stream(comm=comm, extra={"client": client})
  File "/home/qiahan/.conda/envs/firstEnv/lib/python3.9/site-packages/distributed/core.py", line 975, in handle_stream
    msgs = await comm.r

CancelledError: ('<this-array>-predictFlux-c7ce78a0200fe60acd43cf99f4f6eaef', 4, 0, 2, 0)

In [40]:
result1

<xarray.DataArray 'LEH' (time: 17000, latitude: 101, longitude: 101,
                         variable_out: 7)>
array([[[[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan]],

        [[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
...
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan]],

        [[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan]]]])
Coordinates:
  * variable_out  (variable_out) int64 0 1 2 3 4 5 6
  * time          (time) datetime64[ns] 2015-01-01T01:00:00 ... 2015-12-21T04...
  * latitude      (latitude) float64 55.0 54.9 54.8 54.7 ... 45.3 45.2 45.1 45.0
  * longitude     (longitude) float64 2.0 2.1 2.2 2.3 ... 11.7 11.8 11.9 12.0

## export the result